In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
from PIL import Image
from pathlib import Path
import re
import numpy as np
import decimal

In [ ]:
def read_lores_img_set(img_set_path, n_leds, **kwargs):
    """Reads the files of the Lo-Res image set and returns all of them
    as a 3-dimentional array (numpy.ndarray).
    """
    img_prefix = kwargs.pop('prefix', 'IMG_')
    img_format = kwargs.pop('format', 'TIF')
    p = Path(img_set_path)
    str_n_leds = str(n_leds)
    re_wide_num_range = '[0-' + str_n_leds[0] + ']'
    for i in range(len(str_n_leds) - 1):
         re_wide_num_range += '[0-9]'
    # Creates a list with the files within p (Path) whose name begins
    # with glob_pattern (string). glob() does not support regexp.
    glob_pattern = img_prefix + re_wide_num_range + '.*'
    file_list = list(p.glob(glob_pattern))
    file_list.sort()  # Sort the file list.
    # Create a regexp and use it to filter file_list (list).
    str_regex = img_prefix + '(?!0+\.)'
    if len(str_n_leds) > 1:
        str_regex += '(?!' + str_n_leds[0]
        for i in range(1, len(str_n_leds)):
            str_regex += '[' + str(int(str_n_leds[i]) + 1) + '-9]'
        str_regex += ')'
    str_regex += re_wide_num_range
    str_regex += '.('+ img_format.lower() + '|'
    str_regex += img_format.upper() + ')'
    filtered_file_list = list()
    img_cube = list()
    # Convert from regexp pattern to regexp object.
    regexp = re.compile(str_regex)
    # Store the images as 2-dimensionals arrays within a 3-dimensional
    # array.
    for item in file_list:
        m = regexp.search(str(item))
        if m and str(item).endswith(m.group(0)):
            img_path = str(item)
            img_num = m.group(0).split('_')[1].split('.')[0]
            print('<FP-ImageRead-' + img_num + '>')
            img = Image.open(img_path).convert('L')
            arr_img = np.array(img)
            # If img.close() is before the conversion, arr_img
            # (numpy.ndarray) will contain TiffImageFile elements.
            img.close()
            img_cube.append(arr_img)
    return np.array(img_cube)

In [ ]:
def round_half_up(num):
    """In Python 3, the round() function had changed from Python 2.
    For example, in Python 3, round(2.5) returns 2 (int) such
    round(1.5). For that case, it is possible to get 3 (int) instead 2
    using the code below [1].
    
    [1] Barthelemy. (2014). Python 3.x rounding behavior. Message
    posted to
    http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
    """
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))